In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv')

In [3]:
df.RainTomorrow.replace({'Yes':1, 'No':0}, inplace=True)
df.RainToday.replace({'Yes':1, 'No':0}, inplace=True)
df.Date = pd.to_datetime(df.Date)
df.dropna(subset='RainTomorrow', inplace=True)

In [4]:
for col in df.select_dtypes('float64'):
    df[col] = pd.to_numeric(df[col], downcast = 'float')

In [5]:
df.dtypes

Date             datetime64[ns]
Location                 object
MinTemp                 float32
MaxTemp                 float32
Rainfall                float32
Evaporation             float32
Sunshine                float32
WindGustDir              object
WindGustSpeed           float32
WindDir9am               object
WindDir3pm               object
WindSpeed9am            float32
WindSpeed3pm            float32
Humidity9am             float32
Humidity3pm             float32
Pressure9am             float32
Pressure3pm             float32
Cloud9am                float32
Cloud3pm                float32
Temp9am                 float32
Temp3pm                 float32
RainToday               float32
RainTomorrow            float32
dtype: object

In [6]:
X = df.select_dtypes('float')
y = df.RainTomorrow

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
imputer = SimpleImputer(strategy='median')
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasatt

In [9]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [10]:
generator_train = TimeseriesGenerator(X_train, y_train, length=1, batch_size=1)
generator_test = TimeseriesGenerator(X_test, y_test, length=1, batch_size=1)

In [11]:
model = Sequential()
model.add(LSTM(32, input_shape=X_train.shape[1:], return_sequences=True))
model.add(LSTM(32))

model.add(Dense(1, activation='sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[BinaryAccuracy()])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 18, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 12705 (49.63 KB)
Trainable params: 12705 (49.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
filepath = "weights_32_2layers.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')

In [15]:
import tensorflow as tf
print(tf.test.gpu_device_name())
# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

/device:GPU:0


In [16]:
with tf.device('/gpu:0'):
    model.fit(X_train, y_train, batch_size=32, epochs=200, verbose=0, validation_split=0.2,
             callbacks=[checkpoint])


Epoch 1: loss improved from inf to 0.05472, saving model to weights_32_2layers.hdf5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: loss improved from 0.05472 to 0.00018, saving model to weights_32_2layers.hdf5

Epoch 3: loss improved from 0.00018 to 0.00004, saving model to weights_32_2layers.hdf5

Epoch 4: loss improved from 0.00004 to 0.00001, saving model to weights_32_2layers.hdf5

Epoch 5: loss improved from 0.00001 to 0.00000, saving model to weights_32_2layers.hdf5

Epoch 6: loss improved from 0.00000 to 0.00000, saving model to weights_32_2layers.hdf5

Epoch 7: loss improved from 0.00000 to 0.00000, saving model to weights_32_2layers.hdf5

Epoch 8: loss improved from 0.00000 to 0.00000, saving model to weights_32_2layers.hdf5

Epoch 9: loss improved from 0.00000 to 0.00000, saving model to weights_32_2layers.hdf5

Epoch 10: loss improved from 0.00000 to 0.00000, saving model to weights_32_2layers.hdf5

Epoch 11: loss improved from 0.00000 to 0.00000, saving model to weights_32_2layers.hdf5

Epoch 12: loss improved from 0.00000 to 0.00000, saving model to weights_32_2layers.hdf5

Epoch 13: loss im

In [17]:
loss, acc = model.evaluate(X_test, y_test)

1467/1467 [==============================] - 4s 3ms/step - loss: 2.4602e-09 - binary_accuracy: 1.0000


In [18]:
loss, acc

(2.460221137923213e-09, 1.0)